In [1]:
# Sort based on match scores, 
# 1. cutoff, 2. save file with annotation
import collections

import compress_pickle
import itertools
import libsbml
import numpy as np
import os
import pickle
import pandas as pd
import sys
import time
import matplotlib.pyplot as plt
%matplotlib inline  

BIOMD_12 = 'BIOMD0000000012.xml'
BASE_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/'
DATA_DIR = os.path.join(BASE_DIR, "DATA")
ALGO_DIR = os.path.join(DATA_DIR, "algo")
CHEBI_DIR = os.path.join(DATA_DIR, "chebi")
RHEA_DIR = os.path.join(DATA_DIR, "rhea")
BIOMODEL_DIR = os.path.join(DATA_DIR, "biomodels/curated_biomodels_31mar2021")
BIGG_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/DATA/bigg'
ecoli_fpath = os.path.join(BIGG_DIR, "e_coli_core.xml")

PROJ_DIR = os.path.join(os.getcwd(), os.pardir)
AMAS_DIR = os.path.join(PROJ_DIR, "AMAS")
sys.path.append(PROJ_DIR)

SUPPL_DIR = os.path.join(PROJ_DIR, os.pardir, "AMAS_suppl")
ACCURACY_DIR = os.path.join(SUPPL_DIR, "data_for_credibility")
RESULT_DIR = '/Users/woosubs/Desktop/AutomateAnnotation/AMAS_suppl/result_files'

from AMAS import species_annotation as sa
from AMAS import reaction_annotation as ra
from AMAS import recommender
from AMAS import constants as cn
from AMAS import iterator as it
from AMAS import tools

In [2]:
E_COLI_PATH = os.path.join(cn.TEST_DIR, 'e_coli_core.xml')
recom = recommender.Recommender(libsbml_fpath=E_COLI_PATH)
reader = libsbml.SBMLReader()
document = reader.readSBML(E_COLI_PATH)
model = document.getModel()

In [4]:
exist_anot_species = list(recom.species.exist_annotation_formula)
exist_anot_reaction = list(recom.reactions.exist_annotation)

In [15]:

one_rid = exist_anot_reaction[1]
inp_recom = recom.getReactionAnnotation(pred_id=one_rid)
inp_recom

Recommendation(id='R_PFL', credibility=0.92, candidates=[('RHEA:11844', 1.0), ('RHEA:17425', 0.8), ('RHEA:22988', 0.8), ('RHEA:22992', 0.8), ('RHEA:28042', 0.8), ('RHEA:12765', 0.667), ('RHEA:21912', 0.667), ('RHEA:44140', 0.667)], urls=['https://www.rhea-db.org/rhea/11844', 'https://www.rhea-db.org/rhea/17425', 'https://www.rhea-db.org/rhea/22988', 'https://www.rhea-db.org/rhea/22992', 'https://www.rhea-db.org/rhea/28042', 'https://www.rhea-db.org/rhea/12765', 'https://www.rhea-db.org/rhea/21912', 'https://www.rhea-db.org/rhea/44140'], labels=['formate C-acetyltransferase activity', 'pyruvate dehydrogenase (NADP+) activity', 'malonate-semialdehyde dehydrogenase (acetylating) activity', 'malonate-semialdehyde dehydrogenase (acetylating, NAD+) activity', 'pyruvate dehydrogenase (NAD+) activity', 'pyruvate synthase activity', 'pyruvate oxidase (CoA-acetylating) activity', 'pyruvate-flavodoxin oxidoreductase activity'])

In [22]:
for one_rid in exist_anot_reaction:
  inp_recom = recom.getReactionAnnotation(pred_id=one_rid) 
  # first, get index, so that greater or equal
  inp_thresh = 0.8
  thresh_index = np.sum([val[1]>=inp_thresh for val in inp_recom.candidates])
  if thresh_index > 0:
    filt_recom = cn.Recommendation(inp_recom.id,
                                   inp_recom.credibility,
                                   inp_recom.candidates[:thresh_index],
                                   inp_recom.urls[:thresh_index],
                                   inp_recom.labels[:thresh_index]
                                  )
    print(filt_recom.id, "Filterd")
  else:
    print(filt_recom.id, "None met the threshold!")
    break

R_PDH None met the threshold!


In [29]:
def filterRecommendationByThreshold(inp_recom, inp_thresh=0.0):
  thresh_index = np.sum([val[1]>=inp_thresh for val in inp_recom.candidates])
  if thresh_index > 0:
    filt_recom = cn.Recommendation(inp_recom.id,
                                   inp_recom.credibility,
                                   inp_recom.candidates[:thresh_index],
                                   inp_recom.urls[:thresh_index],
                                   inp_recom.labels[:thresh_index])
    return filt_recom
  else:
    return None

In [30]:
exist_anot_reaction[1]

'R_PFL'

In [31]:
one_rid = 'R_PFK'
two_rid = 'R_PFL'

In [34]:
one_recom = recom.getReactionAnnotation(pred_id=one_rid)
filterRecommendationByThreshold(inp_recom=one_recom, inp_thresh=0.8)

In [35]:
two_recom = recom.getReactionAnnotation(pred_id=two_rid)
filterRecommendationByThreshold(inp_recom=two_recom, inp_thresh=0.8)

Recommendation(id='R_PFL', credibility=0.92, candidates=[('RHEA:11844', 1.0), ('RHEA:17425', 0.8), ('RHEA:22988', 0.8), ('RHEA:22992', 0.8), ('RHEA:28042', 0.8)], urls=['https://www.rhea-db.org/rhea/11844', 'https://www.rhea-db.org/rhea/17425', 'https://www.rhea-db.org/rhea/22988', 'https://www.rhea-db.org/rhea/22992', 'https://www.rhea-db.org/rhea/28042'], labels=['formate C-acetyltransferase activity', 'pyruvate dehydrogenase (NADP+) activity', 'malonate-semialdehyde dehydrogenase (acetylating) activity', 'malonate-semialdehyde dehydrogenase (acetylating, NAD+) activity', 'pyruvate dehydrogenase (NAD+) activity'])

In [38]:
print(len(two_recom.candidates))
print(len(filterRecommendationByThreshold(inp_recom=two_recom, inp_thresh=0.8).candidates))

8
5


### Include filtered recommendations to update results

In [ ]:
# how to save recommendation? 
# If species, create 

In [19]:
name_lengths = []
comp_numbers = []
exist_anot_species = list(recom.species.exist_annotation_formula)
exist_anot_reaction = list(recom.reactions.exist_annotation)
for one_sid in exist_anot_species:
  used_name = recom.species.getNameToUse(inp_id=one_sid)
  name_lengths.append(len(used_name))
for one_rid in exist_anot_reaction:
  comp_numbers.append(len(recom.reactions.reaction_components[one_rid]))
print(np.mean(name_lengths))
print(np.mean(comp_numbers))

14.541666666666666
3.953846153846154


In [29]:
biomd_spec_res = compress_pickle.load(os.path.join(RESULT_DIR, 'biomd_spec_res.lzma'))
biomd_reac_res = compress_pickle.load(os.path.join(RESULT_DIR, 'biomd_reac_res.lzma'))
bigg_spec_res = compress_pickle.load(os.path.join(RESULT_DIR, 'bigg_spec_res.lzma'))
bigg_reac_res = compress_pickle.load(os.path.join(RESULT_DIR, 'bigg_reac_res.lzma'))

In [30]:
spec_biomds = list(biomd_spec_res['biomd_anotspec_numbers'].keys())
reac_biomds = list(biomd_reac_res['biomd_anotreac_numbers'].keys())
spec_biggs = list(bigg_spec_res['bigg_anotspec_numbers'].keys())
reac_biggs = list(bigg_reac_res['bigg_anotreac_numbers'].keys())

## Name Length of Species

In [53]:
one_sid

'E1'

In [55]:
recom.species.names

{'G1': 'Glucose_1', 'T1': 'ATP_1', 'G2': 'Glucose_2', 'T2': 'ATP_2'}

In [58]:
# species 
splen_bio = []
nulls_bio = []
for idx, one_biomd in enumerate(spec_biomds):
  if idx % 50 == 0:
    print("We are at %d." % idx)
  one_fpath = os.path.join(BIOMODEL_DIR, one_biomd)
  recom = recommender.Recommender(libsbml_fpath=one_fpath)
  name_lengths = []
  null_display_name = 0
  exist_anot_species = list(recom.species.exist_annotation_formula)
  for one_sid in exist_anot_species:
    used_name = recom.species.getNameToUse(inp_id=one_sid)
    name_lengths.append(len(used_name))
    if recom.species.names[one_sid] == '':
      null_display_name += 1
  splen_bio.append(np.mean(name_lengths))
  nulls_bio.append(null_display_name / len(name_lengths))

print("Biomodel species", np.mean(splen_bio))
print("Biomodel, null display names", np.mean(nulls_bio))


splen_bigg = []
nulls_bigg = []
for idx, one_bigg in enumerate(spec_biggs):
  if idx % 10 == 0:
    print("We are at %d." % idx)
  one_fpath = os.path.join(BIGG_DIR, one_bigg)
  recom = recommender.Recommender(libsbml_fpath=one_fpath)
  name_lengths = []
  null_display_name = 0
  exist_anot_species = list(recom.species.exist_annotation_formula)
  for one_sid in exist_anot_species:
    used_name = recom.species.getNameToUse(inp_id=one_sid)
    name_lengths.append(len(used_name))
    if recom.species.names[one_sid] == '':
      null_display_name += 1
  splen_bigg.append(np.mean(name_lengths))
  nulls_bigg.append(null_display_name / len(name_lengths))

print("BiGG species", np.mean(splen_bigg))
print("BiGG, null display names", np.mean(nulls_bigg))

We are at 0.
We are at 50.
We are at 100.
We are at 150.
We are at 200.
We are at 250.
Biomodel species 7.205431397239302
Biomodel, null display names 0.1975190165111077
We are at 0.
We are at 10.
We are at 20.
We are at 30.
We are at 40.
We are at 50.
We are at 60.
We are at 70.
We are at 80.
We are at 90.
We are at 100.
BiGG species 19.301197015646924
BiGG, null display names 0.0


## Num Components of Reactions

In [44]:
# reactions 
numcom_bio = []
for idx, one_biomd in enumerate(reac_biomds):
  if idx % 50 == 0:
    print("We are at %d." % idx)
  one_fpath = os.path.join(BIOMODEL_DIR, one_biomd)
  recom = recommender.Recommender(libsbml_fpath=one_fpath)
  num_comps = []
  exist_anot_reactions = list(recom.reactions.exist_annotation)
  for one_rid in exist_anot_reactions:
    numc = recom.reactions.reaction_components[one_rid]
    num_comps.append(len(numc))
  numcom_bio.append(np.mean(num_comps))

print("Biomodel Reactions", np.mean(numcom_bio))

numcom_bigg = []
for idx, one_bigg in enumerate(reac_biggs):
  if idx % 10 == 0:
    print("We are at %d." % idx)
  one_fpath = os.path.join(BIGG_DIR, one_bigg)
  recom = recommender.Recommender(libsbml_fpath=one_fpath)
  num_comps = []
  exist_anot_reactions = list(recom.reactions.exist_annotation)
  for one_rid in exist_anot_reactions:
    numc = recom.reactions.reaction_components[one_rid]
    num_comps.append(len(numc))
  numcom_bigg.append(np.mean(num_comps))

print("BiGG Reactions", np.mean(numcom_bigg))

We are at 0.
We are at 50.
We are at 100.
Biomodel Reactions 2.6807023403128025
We are at 0.
We are at 10.
We are at 20.
We are at 30.
We are at 40.
We are at 50.
We are at 60.
We are at 70.
We are at 80.
We are at 90.
We are at 100.
BiGG Reactions 4.1314935172461835


In [38]:
numc

['M_thr__L_c', 'M_h_e', 'M_h_c', 'M_thr__L_e']

In [14]:
err_rid = 'R_GLNabc'
err_r = model.getReaction(err_rid)
print(err_r.getAnnotationString())
pred_err_r = recom.getReactionAnnotation(pred_id=err_rid)
pred_err_r.candidates

<sbml:annotation xmlns:sbml="http://www.sbml.org/sbml/level3/version1/core">
  <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#">
    <rdf:Description rdf:about="#R_GLNabc">
      <bqbiol:is xmlns:bqbiol="http://biomodels.net/biology-qualifiers/">
        <rdf:Bag>
          <rdf:li rdf:resource="http://identifiers.org/bigg.reaction/GLNabc"/>
          <rdf:li rdf:resource="http://identifiers.org/biocyc/META:ABC-12-RXN"/>
          <rdf:li rdf:resource="http://identifiers.org/ec-code/3.6.3.21"/>
          <rdf:li rdf:resource="http://identifiers.org/metanetx.reaction/MNXR100258"/>
          <rdf:li rdf:resource="http://identifiers.org/rhea/29895#1"/>
          <rdf:li rdf:resource="http://identifiers.org/rhea/29897#1"/>
          <rdf:li rdf:resource="http://identifiers.org/rhea/29898#1"/>
          <rdf:li rdf:resource="http://identifiers.org/rhea/29896#1"/>
          <rdf:li rdf:resource="http://identifiers.org/seed.reaction/rxn11233"/>
          <rdf:li rdf:resource="

[('RHEA:19425', 0.833), ('RHEA:31599', 0.714)]

In [62]:
one_biomd = 'BIOMD0000000012.xml'
one_fpath = os.path.join(BIOMODEL_DIR, one_biomd)
recom = recommender.Recommender(libsbml_fpath=one_fpath)
reader = libsbml.SBMLReader()
document = reader.readSBML(one_fpath)
model = document.getModel()

In [86]:
def getStr(inp_r):
  if len(inp_r)==0:
    return ""
  else:
    ls_str = ["%s * %s" % (str(val.stoichiometry), val.species) for val in inp_r]
    return "+".join(ls_str)
def getReactionStr(inp_reaction):
  rs = inp_reaction.getListOfReactants()
  ps = inp_reaction.getListOfProducts()
  return getStr(rs) + " => " + getStr(ps) 

In [88]:
one_reaction = model.getReaction(0)

for one_r in model.getListOfReactions():
  one_str = getReactionStr(one_r)
  print(one_str)
  print(one_r.getAnnotationString)

1.0 * X => 
<bound method SBase.getAnnotationString of <Reaction Reaction1 "degradation of LacI transcripts">>
1.0 * Y => 
<bound method SBase.getAnnotationString of <Reaction Reaction2 "degradation of TetR transcripts">>
1.0 * Z => 
<bound method SBase.getAnnotationString of <Reaction Reaction3 "degradation of CI transcripts">>
 => 1.0 * PX
<bound method SBase.getAnnotationString of <Reaction Reaction4 "translation of LacI">>
 => 1.0 * PY
<bound method SBase.getAnnotationString of <Reaction Reaction5 "translation of TetR">>
 => 1.0 * PZ
<bound method SBase.getAnnotationString of <Reaction Reaction6 "translation of CI">>
1.0 * PX => 
<bound method SBase.getAnnotationString of <Reaction Reaction7 "degradation of LacI">>
1.0 * PY => 
<bound method SBase.getAnnotationString of <Reaction Reaction8 "degradation of TetR">>
1.0 * PZ => 
<bound method SBase.getAnnotationString of <Reaction Reaction9 "degradation of CI">>
 => 1.0 * X
<bound method SBase.getAnnotationString of <Reaction Reaction

In [91]:
for one_s in model.getListOfSpecies():
  print(one_s.getId())
  print(one_s.name)

PX
LacI protein
PY
TetR protein
PZ
cI protein
X
LacI mRNA
Y
TetR mRNA
Z
cI mRNA
